In [1]:
!nvidia-smi
# We import all our dependencies.
from n2v.models import N2VConfig, N2V
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from csbdeep.utils import plot_history,plot_some
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from matplotlib import pyplot as plt
import urllib
import os
import zipfile

Wed Jul  7 15:25:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 450.57       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:17:00.0  On |                  Off |
| 33%   38C    P8    28W / 260W |  22984MiB / 24211MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     Off  | 00000000:73:00.0 Off |                  Off |
| 65%   

In [ ]:
BaseDir =  '/home/sancere/Kepler/CurieTrainingDatasets/ClaudiaTomatoDenoising/Tomato_Denoising/'

Model_Dir = '/home/sancere/Kepler/CurieDeepLearningModels/ClaudiaTomatoDenoising/'
Model_Name = 'N2VTomato'

datagen = N2V_DataGenerator()

imgs = datagen.load_imgs_from_directory(directory = BaseDir , dims='ZYX')
plt.figure(figsize=(32,16))
plt.imshow(np.max(imgs[0][0,...,0],axis=0), 
           cmap='magma',
           vmin=np.percentile(imgs[0],0.1),
           vmax=np.percentile(imgs[0],99.9)
          )
plt.show()





(13, 1024, 1024) /home/sancere/Kepler/CurieTrainingDatasets/ClaudiaTomatoDenoising/Tomato_Denoising/20210325_124A_N1TOM_day2_region1-1.tif


In [ ]:
patch_shape = (8, 512, 512)
Epochs = 20
NetworkDepth = 3
batch_size = 1
patches = datagen.generate_patches_from_list(imgs[:1], shape=patch_shape)
X = patches[:30]
X_val = patches[30:]

# Let's look at two patches.
plt.figure(figsize=(14,7))
plt.subplot(1,2,1)
plt.imshow(X[0,1,...,0],cmap='magma')
plt.title('Training Patch');
plt.subplot(1,2,2)
plt.imshow(X_val[0,1,...,0],cmap='magma')
plt.title('Validation Patch');

In [ ]:
config = N2VConfig(X, unet_kern_size=3, unet_n_depth = NetworkDepth,
                   train_steps_per_epoch=400,train_epochs= Epochs, train_loss='mse', batch_norm=True, 
                   train_batch_size=batch_size, n2v_perc_pix=0.198,n2v_patch_shape= patch_shape, 
                   n2v_manipulator='uniform_withCP', n2v_neighborhood_radius=4)

# Let's look at the parameters stored in the config-object.
vars(config)
model = N2V(config=config, name=Model_Name, basedir=Model_Dir)

In [ ]:

history = model.train(X, X_val)

In [ ]:
print(sorted(list(history.history.keys())))
plt.figure(figsize=(16,5))
plot_history(history,['loss','val_loss']);

In [ ]:
plt.figure(figsize=(25,7))
_P = model.keras_model.predict(X_val[:5])

plot_some(X_val[:5],_P,pmax=99.5)
plt.suptitle('5 example validation patches\n'      
             'top row: input (source),  '          
             'middle row: denoised),  '
             );